<a href="https://colab.research.google.com/github/Beloveddream/RS/blob/main/LSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
pip install datasketch

     |████████████████████████████████| 71kB 8.5MB/s 


In [4]:
from datasketch import MinHash
data1 = ['这个', '程序', '代码', '太乱', '那个', '代码', '规范']
data2 = ['这个', '程序', '代码', '不', '规范', '那个', '更', '规范']


m1 = MinHash()
m2 = MinHash()
for d in data1:
	m1.update(d.encode('utf8'))
for d in data2:
    m2.update(d.encode('utf8'))
print("使用MinHash预估的Jaccard相似度", m1.jaccard(m2))

s1 = set(data1)
s2 = set(data2)
actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
print("Jaccard相似度实际值", actual_jaccard)
print(s1.intersection(s2))
print(len(s1.intersection(s2)))
print(len(s1.union(s2)))
print(s1.union(s2))

使用MinHash预估的Jaccard相似度 0.6015625
Jaccard相似度实际值 0.625
{'代码', '程序', '规范', '那个', '这个'}
5
8
{'太乱', '代码', '程序', '不', '规范', '更', '那个', '这个'}


In [7]:
from datasketch import MinHash, MinHashLSH
data1 = ['这个', '程序', '代码', '太乱', '那个', '代码', '规范']
data2 = ['这个', '程序', '代码', '不', '规范', '那个', '更', '规范']
data3 = ['这个', '程序', '代码', '不', '规范', '那个', '规范', '些']

# 创建MinHash对象
m1 = MinHash()
m2 = MinHash()
m3 = MinHash()
for d in data1:
	m1.update(d.encode('utf8'))
for d in data2:
	m2.update(d.encode('utf8'))
for d in data3:
	m3.update(d.encode('utf8'))
# 创建LSH
lsh = MinHashLSH(threshold=0.6, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m2)
print("近似的邻居（Jaccard相似度>0.5）", result)

近似的邻居（Jaccard相似度>0.5） ['m3', 'm2']


In [10]:
pip install simhash

In [13]:
# -*- coding: utf-8 -*-
from sklearn.feature_extraction.text import TfidfVectorizer
from simhash import Simhash, SimhashIndex

print(Simhash('这个程序代码太乱').value)
#print(Simhash('1').distance(Simhash('2')))
#sh1 = Simhash(u'这个程序代码太乱,那个代码规范')
#sh2 = Simhash(u'这个程序代码不规范,那个更规范')
#print(sh1.value)
#print(sh2.value)
#print(sh1.distance(sh2))

data = [
    '这个程序代码太乱,那个代码规范',
    '这个程序代码不规范,那个更规范',
    '我是佩奇，这是我的弟弟乔治'
]

data = [
    '这个 程序 代码 太乱 那个 代码 规范',
    '这个 程序 代码 不 规范 那个 更 规范',
    '我 是 佩奇 这 是 我的 弟弟 乔治'
]

vec = TfidfVectorizer()
D = vec.fit_transform(data)
voc = dict((i, w) for w, i in vec.vocabulary_.items())
#print(voc)

# 生成Simhash
sh_list = []
for i in range(D.shape[0]):
    Di = D.getrow(i)
    #print(Di.indices)
    #print(Di.data)
    # features表示 (token, weight)元祖形式的列表
    features = zip([voc[j] for j in Di.indices], Di.data)
    sh_list.append(Simhash(features))
print(sh_list[0].distance(sh_list[1]))
print(sh_list[0].distance(sh_list[2]))
print(sh_list[1].distance(sh_list[2]))


10845600900754665595
15
32
37


In [14]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
data1 = ['这个', '程序', '代码', '太乱', '那个', '代码', '规范']
data2 = ['这个', '程序', '代码', '不', '规范', '那个', '更', '规范']
data3 = ['这个', '程序', '代码', '不', '规范', '那个', '规范', '些']

# 创建MinHash对象
m1 = MinHash()
m2 = MinHash()
m3 = MinHash()
for d in data1:
	m1.update(d.encode('utf8'))
for d in data2:
	m2.update(d.encode('utf8'))
for d in data3:
	m3.update(d.encode('utf8'))
# 创建LSH Forest
forest = MinHashLSHForest()
forest.add("m2", m2)
forest.add("m3", m3)
# 在检索前，需要使用index
forest.index()
# 判断forest是否存在m2, m3
print("m2" in forest)
print("m3" in forest)
# 查询forest中与m1相似的Top-K个邻居
result = forest.query(m1, 2)
print("Top 2 邻居", result)

True
True
Top 2 邻居 ['m3', 'm2']


In [15]:
from datasketch import MinHash, MinHashLSH, MinHashLSHEnsemble
data1 = ['这个', '程序', '代码', '太乱', '那个', '代码', '规范']
data2 = ['这个', '程序', '代码', '不', '规范', '那个', '更', '规范']
data3 = ['这个', '程序', '代码', '不', '规范', '那个', '规范', '些']

# 创建MinHash对象
m1 = MinHash()
m2 = MinHash()
m3 = MinHash()
for d in data1:
	m1.update(d.encode('utf8'))
for d in data2:
	m2.update(d.encode('utf8'))
for d in data3:
	m3.update(d.encode('utf8'))
# 创建LSH Ensemble
lshensemble = MinHashLSHEnsemble(threshold=0.8, num_perm=128)
# Index takes an iterable of (key, minhash, size)
lshensemble.index([("m2", m2, len(data2)), ("m3", m3, len(data3))])
# 判断lshensemble是否存在m2, m3
print("m2" in lshensemble)
print("m3" in lshensemble)
# 查询与m1相似度大于0.8的集合
print("与m1相似度大于0.8的集合：")
for key in lshensemble.query(m1, len(data1)):
    print(key)

True
True
与m1相似度大于0.8的集合：
m3
m2


In [16]:
# 对天猫双11新闻进行相似句子Top-K查询
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [17]:
# 读取文件
f = open('/content/gdrive/My Drive/LSH/sentences.txt', 'r', encoding='UTF-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentences = re.split('[。！？]', text.replace('\n', ''))

In [18]:
# 最后一行如果为空，则删除
if sentences[len(sentences)-1] == '':
    sentences.pop()
print(sentences)
print(len(sentences))

['11月10日深夜到11日凌晨，杭州阿里巴巴园区光柱擎天、灯火辉煌、不眠不休', '这是第11年双11，一个稀疏平常的日子凭空成为全民购物狂欢本就不可思议，而更不可思议的是不断刷新的消费数据', '仅用时96秒，2019天猫双11总成交额便超过100亿元，这个速度，比2018年缩短了29秒', '随后，500亿元、1000亿元，每一个成交额小目标都以更快的速度实现', '截至中证君发稿，2019天猫双11前三小时总成交额达到1325.7亿元，已经超过2018年天猫双11全天2135亿元总成交额的62%', '2018年，阿里巴巴用15小时49分钟39秒时间，打破了2017年1682亿元的双11全天成交纪录，最终以27%的增速定格在2135亿元', '今年双11，阿里巴巴又将以怎样的增速、多长时间来打破自己的纪录，它能否一举突破3000亿元大关，同时提振全国内需', '作为双11购物狂欢盛典的鼻祖，阿里巴巴在过去三年双11全网成交额中占据三分之二天下，京东、苏宁以及近期势头凶猛的拼多多均处于“后来”但尚未“居上”状态', '首先来看阿里巴巴（天猫）战报', '据介绍，今年，超过1000万款商品登陆天猫双11，超100万款新品在天猫双11首发，来自78个国家和地区的跨境电商进口商品、全国1000个数字农业基地的高品质农产品、2000个传统制造业产业带的工厂直供好货，以新供给满足消费者的多元化需求', '00:01:36，2019天猫双11总成交额超100亿元', '2018年用时2分05秒，今年缩短29秒', '00:12:49，2019天猫双11总成交额超500亿元', '2018年用时26分3秒，今年缩短一半', '01:03:59，2019天猫双11总成交额超1000亿元', '比2018年快了43分钟27秒，比2017年快了将近8小时']
16


In [22]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

# 设置停用词
#stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]
stop = []
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

query = '00:01:36，2019天猫双11总成交额超100亿元'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.914 seconds.
Prefix dict has been built successfully.


10 1.0 00:01:36，2019天猫双11总成交额超100亿元
12 0.8125 00:12:49，2019天猫双11总成交额超500亿元
14 0.9375 01:03:59，2019天猫双11总成交额超1000亿元
Top 3 邻居 [10, 12, 14]


In [23]:
# 对微博新闻进行相似句子Top-K查询
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [24]:
# 读取文件
f = open('/content/gdrive/My Drive/LSH/weibos.txt', 'r', encoding='UTF-8')
text = f.read()
# 以标点符号作为分隔，去掉\n换行符号
sentences = re.split('[。！？# ……]', text.replace('\n', ' '))

# 最后一行如果为空，则删除
if sentences[len(sentences)-1] == '':
    sentences.pop()

# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            item_str += " "
    return item_str
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

# 设置停用词
#stop = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
stop = []
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

query = '中国足协：接受里皮辞职请求，将深刻反思'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)
print('Query:',query)
# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

Query: 中国足协：接受里皮辞职请求，将深刻反思
10 0.3046875 据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决
43 1.0 中国足协：接受里皮辞职请求，将深刻反思
6 0.0703125 谁将成为新主帅，成为广大球迷关注的焦点
Top 3 邻居 [10, 43, 6]
